## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [ ]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 
            'BusinessTravel', 
            'DistanceFromHome', 
            'Education', 
            'EducationField', 
            'EnvironmentSatisfaction', 
            'HourlyRate', 
            'JobInvolvement',
            'JobLevel',
            'JobRole',
            'JobSatisfaction',
            'MaritalStatus',
            'NumCompaniesWorked',
            'OverTime',
            'PercentSalaryHike',
            'PerformanceRating',
            'RelationshipSatisfaction',
            'StockOptionLevel',
            'TotalWorkingYears',
            'TrainingTimesLastYear',
            'WorkLifeBalance',
            'YearsAtCompany',
            'YearsInCurrentRole',
            'YearsSinceLastPromotion',
            'YearsWithCurrManager']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [ ]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

cols_to_encode = ['BusinessTravel', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime']

# Determine the number of value counts in each column.
for col in cols_to_encode:
    print(f'{col}: {X_train[col].value_counts()}\n')


In [ ]:
# Use OneHotEncoder to encode the columns
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')


# Fit and encode the training data
X_train_encoded = encoder.fit_transform(X_train[cols_to_encode])
X_test_encoded = encoder.transform(X_test[cols_to_encode])

# Convert the encoded arrays to DataFrames
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(cols_to_encode))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(cols_to_encode))

print(X_train_encoded_df.dtypes)
print(X_test_encoded_df.dtypes)


In [ ]:
X_train.columns

In [ ]:
# Create a list of columns to scale
cols_to_scale = ['Age', 
                'DistanceFromHome',
                'Education',
                'EnvironmentSatisfaction', 
                'HourlyRate',
                'JobInvolvement',
                'JobLevel',
                'JobSatisfaction', 
                'NumCompaniesWorked', 
                'PercentSalaryHike',
                'PerformanceRating',
                'RelationshipSatisfaction',
                'StockOptionLevel', 
                'TotalWorkingYears', 
                'TrainingTimesLastYear',
                'WorkLifeBalance', 
                'YearsAtCompany', 
                'YearsInCurrentRole', 
                'YearsSinceLastPromotion', 
                'YearsWithCurrManager']

In [ ]:
# Create a StandardScaler
scale = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scale.fit_transform(X_train[cols_to_scale])


# Scale the training and testing data
X_test_scaled = scale.transform(X_test[cols_to_scale])

# Convert the scaled arrays to DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=cols_to_scale)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=cols_to_scale)

X_train_scaled_df.head()

In [ ]:
# Concatenate the scaled and encoded columns
X_train_df = pd.concat([X_train_encoded_df, X_train_scaled_df], axis=1)
X_test_df = pd.concat([X_test_encoded_df, X_test_scaled_df], axis=1)

In [ ]:
y_df['Department'].value_counts()

In [ ]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
dept_encoder.fit(np.array(y_train['Department'].values.reshape(-1, 1)))

# Create two new variables by applying the encoder
# to the training and testing data
y_dept_train = dept_encoder.transform(np.array(y_train['Department'].values.reshape(-1, 1)))
y_dept_test = dept_encoder.transform(np.array(y_test['Department'].values.reshape(-1, 1)))



In [ ]:
y_df['Attrition'].value_counts()

In [ ]:
# Create a OneHotEncoder for the Attrition column
atr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

# Fit the encoder to the training data
# atr_encoder.fit(np.array(y_train['Attrition'].values.reshape(-1, 1)))
atr_encoder.fit(np.array(y_train['Attrition'].values.reshape(-1, 1)))

# Create two new variables by applying the encoder
# to the training and testing data
y_atr_train = atr_encoder.transform(np.array(y_train['Attrition'].values.reshape(-1, 1)))
y_atr_test = atr_encoder.transform(np.array(y_test['Attrition'].values.reshape(-1, 1)))
y_atr_train


## Create, Compile, and Train the Model

In [ ]:
X_train_df.shape[0]

In [ ]:
# Find the number of columns in the X training data
number_input_features = len(X_train_df.columns)
# number_input_features = X_train_df.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(number_input_features,), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(128, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(64, activation='relu', name='shared_layer2')(shared_layer1)
shared_layer3 = layers.Dense(32, activation='relu', name='shared_layer3')(shared_layer2)

In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden_layer = layers.Dense(16, activation='relu', name='dept_hidden_layer')(shared_layer3)

# Create the output layer
dept_output_layer = layers.Dense(y_dept_train.shape[1], activation='softmax', name='dept_output_layer')(dept_hidden_layer)


In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
atr_hidden_layer = layers.Dense(16, activation='relu', name='atr_hidden_layer')(shared_layer3)

# Create the output layer
atr_output_layer = layers.Dense(y_atr_train.shape[1], activation='sigmoid', name='atr_output_layer')(atr_hidden_layer)


In [ ]:
# Create the model
model = Model(
    inputs=input_layer, 
    outputs=[dept_output_layer, atr_output_layer],
    name='model')

# Compile the model
model.compile(optimizer='adam',
                loss={'dept_output_layer': 'categorical_crossentropy',
                    'atr_output_layer': 'categorical_crossentropy'},
                metrics={'dept_output_layer': 'accuracy',
                    'atr_output_layer': 'accuracy'})

# Summarize the model
model.summary()

In [ ]:
# Train the model
model.fit(
        X_train_df, 
        {'dept_output_layer': y_dept_train, 
        'atr_output_layer': y_atr_train}, 
        epochs=100, 
        batch_size=32,
        validation_split=0.2
        )


In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_df, 
            {'dept_output_layer': y_dept_test, 
            'atr_output_layer': y_atr_test})
test_results

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]*100}")
print(f"Attrition Accuracy: {test_results[1]*100}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy is not the best metric to use on this data given the imbalance identified by the value counts for Department and Attrition features. The model accuracy is      measuring the proportion of correctly predicated instances out of total instances. A higher probability for one class may result in a high accuracy score by simply predicting the majority class, even though the model may not be correctly identifying the minority class.

2. For the dept_output_layer, the softmax activation function was chosen because it is appropriate for multi-class classification problems. The softmax function converts the raw output scores into probabilities that sum to 1, allowing the model to make a probabilistic prediction for each class, and the class with the highest probability is chosen as the predicted class.
For the atr_output_layer, the sigmoid activation function was chosen because it is appropriate for binary classification problems. The sigmoid function outputs a probability value between 0 and 1, which can be interpreted as the likelihood of the positive class. A threshold (usually 0.5) is then used to decide the class label.

3. The model score can be improved through the following ways:
Data Preprocessing - Adjusting the class weights or using resampling techniques to address the imbalance in the minority class. We could also creating or transforming features to provide a better way to capture the underlying patterns in the data.
Model Architecture - Changing the model architecture by adding more layers, increasing the number of neurons in the layers, or using a different activation function.
Hyperparameter Tuning - Changing the hyperparameters such as the learning rate, batch size, optimizer, loss function, and number of epochs.